<a href="https://colab.research.google.com/github/marianomirabal/etl-ibnorca/blob/main/notebooks/etl_ibnorca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# 1) Instala dependencias (solo la primera vez)
!pip install pandas openpyxl sqlalchemy

import pandas as pd
from sqlalchemy import create_engine

# 2) EXTRACT
# (antes de correrlo, sube tu Excel manualmente al entorno: panel izquierdo 📁 → subir archivo)
df_raw = pd.read_excel('/content/Clientes_faltantes.xlsx')
print("Leídas", df_raw.shape[0], "filas y", df_raw.shape[1], "columnas.")

# 3) TRANSFORM
df = df_raw.copy()
df.columns = (
    df.columns
      .str.strip()
      .str.replace(r'\.+','_',regex=True)
      .str.replace(r'\s+','_',regex=True)
)
keywords = ['SRL','S.R.L.', 'S.R.L','SRL.','S R L.','R.L','RL.','S.A','S.A.','SA.','LTDA','LTDA.', 'CORPORACIÓN','COOPERATIVA','GRUPO','SOCIEDAD DE RESPONSABILIDAD LIMITADA','LIMITADA','ASOCIACION','ASOCIACIÓN','BANCO','CLUB',]
df['Tipo de persona'] = df['Column1_razon_social'].apply(
    lambda x: 'Empresa' if any(k in str(x).upper() for k in keywords) else 'Persona'
)
print(df[['Column1_razon_social','Tipo de persona']].head())

# 4) LOAD
engine = create_engine('sqlite:///clientes_ibnorca.db')
df.to_sql('clientes_limpios', engine, if_exists='replace', index=False)
from sqlalchemy import inspect
inspector = inspect(engine)
print("Tablas en SQLite:", inspector.get_table_names())

Leídas 1660 filas y 3 columnas.
              Column1_razon_social Tipo de persona
0                             CAPO         Persona
1                          SALINAS         Persona
2   ALCIDES GUZMAN AYAVIRI AYAVIRI         Persona
3            Altavista Group S.R.L         Empresa
4                             AYMA         Persona
Tablas en SQLite: ['clientes_limpios']


In [31]:
# Primero definimos la función
def adivinar_sector(nombre):
    nombre = str(nombre).lower()
    if any(palabra in nombre for palabra in ["hospital", "clínica", "med", "salud", "bio", "farma", "laboratorio","laboratorios","lab","pharma"]):
        return "SALUD"
    elif any(palabra in nombre for palabra in ["seguridad", "safety", "riesgo", "osha"]):
        return "SEGURIDAD OCUPACIONAL"
    elif any(palabra in nombre for palabra in ["minera", "mining"]):
        return "MINERÍA"
    elif any(palabra in nombre for palabra in ["ingeniería", "mecánica", "industrial"]):
        return "MECÁNICA"
    elif any(palabra in nombre for palabra in ["construct", "construcciones", "obras", "infra", "arquitect","soboce"]):
        return "CONSTRUCCIÓN"
    elif any(palabra in nombre for palabra in ["agro", "alimentos", "granja", "avícola", "leche", "lácteo", "cárnico","imba","kral","cereal","cerveza","cervecería","food"]):
        return "AGROPECUARIO, AGRÍCOLA Y ALIMENTOS"
    elif any(palabra in nombre for palabra in ["gas", "petrol", "glp", "combustible","hidrocarburos","surtidor"]):
        return "HIDROCARBUROS"
    elif any(palabra in nombre for palabra in ["energía", "eléctrica", "solar", "hidroeléctrica","electrificacion","electrificación","energy"]):
        return "ENERGÍA"
    elif any(palabra in nombre for palabra in ["agua", "medio ambiente", "ambiental", "ecología"]):
        return "MEDIO AMBIENTE Y AGUA"
    elif any(palabra in nombre for palabra in ["transporte", "logística", "shipping", "envío"]):
        return "TRANSPORTE"
    elif any(palabra in nombre for palabra in ["sistemas", "software", "informática", "tecnología","tech","techno"]):
        return "TECNOLOGÍAS DE LA INFORMACIÓN Y COMUNICACIÓN"
    elif any(palabra in nombre for palabra in ["norma", "certifica", "ibnorca", "calidad", "iso"]):
        return "INFRAESTRUCTURA DE LA CALIDAD"
    elif any(palabra in nombre for palabra in ["industria", "fábrica", "producción"]):
        return "INDUSTRIA"
    elif any(palabra in nombre for palabra in ["hogar", "doméstico", "electrodoméstico", "mueble"]):
        return "EQUIPAMIENTO DOMÉSTICO Y COMERCIAL"
    elif any(palabra in nombre for palabra in ["servicio", "consult", "asesor","seguro","seguros"]):
        return "SERVICIOS"
    else:
        return "SERVICIOS"  # Por defecto si no se puede adivinar

# Ahora aplicamos la función solo a filas donde Tipo de persona es Empresa
df.loc[df["Tipo de persona"] == "Empresa", "Sector Empresarial"] = df.loc[
    df["Tipo de persona"] == "Empresa", "Column1_razon_social"
].apply(adivinar_sector)

In [32]:
df.to_excel("empresas_actualizado.xlsx", index=False)
